<a href="https://colab.research.google.com/github/zakariyyabrewster/CGCNN_MOF/blob/main/cgcnn_quickdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zakariyyabrewster/CGCNN_MOF

Cloning into 'CGCNN_MOF'...
remote: Enumerating objects: 12457, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 12457 (delta 43), reused 55 (delta 24), pack-reused 12376 (from 3)
Receiving objects: 100% (12457/12457), 314.08 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (139/139), done.
Updating files: 100% (200/200), done.


In [3]:
%cd content/CGCNN_MOF

/content/CGCNN_MOF


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 122.7 MB/s eta 0:00:00


In [5]:
!bash scripts/extract_db.sh # works

Extracting CoRE2019 dataset to test_datasets/CoRE2019
CoRE2019 dataset extracted successfully to test_datasets/CoRE2019


In [6]:
import os
import yaml
import pandas as pd

from finetune_cgcnn import FineTune

In [7]:
config = yaml.load(open("config_ft_cgcnn.yaml", "r"), Loader=yaml.FullLoader)
config['epochs'] = 1
config['batch_size'] = 1
ftf = config['fine_tune_from']
seed = config['random_seed']
target_property = config['target_property']
task_name = config['data_name']
log_dir = os.path.join(
    'training_results/finetuning/CGCNN',
    'CGCNN_{}_{}_{}_{}'.format(ftf, task_name,seed, target_property)
)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
config

{'batch_size': 1,
 'epochs': 1,
 'eval_every_n_epochs': 1,
 'fine_tune_from': 'scratch',
 'log_every_n_steps': 50,
 'gpu': 'cuda:0',
 'random_seed': 1,
 'task': 'regression',
 'optim': {'optimizer': 'Adam',
  'lr': 0.01,
  'momentum': 0.9,
  'weight_decay': '1e-6'},
 'model': {'atom_fea_len': 64,
  'h_fea_len': 512,
  'n_conv': 3,
  'n_h': 1,
  'drop_ratio': 0.2},
 'data_name': 'CoRE2019',
 'target_property': 'Di',
 'dataset': {'root_dir': './test_datasets/CoRE2019',
  'label_dir_template': './test_datasets/id_{target_property}.csv',
  'max_num_nbr': 12,
  'radius': 8,
  'dmin': 0,
  'step': 0.2},
 'dataloader': {'val_ratio': 0.15,
  'test_ratio': 0.15,
  'num_workers': 0,
  'subset_size': 500}}

In [8]:
fine_tune = FineTune(config, log_dir)
fine_tune.train()
loss, metric = fine_tune.test()

fn = 'CGCNN_{}_{}_{}_{}.csv'.format(ftf,task_name,seed, target_property)
df = pd.DataFrame([[loss, metric.item()]])
df.to_csv(
    os.path.join(log_dir, fn),
    mode='a', index=False, header=True
)

Running on: cuda:0
The random seed is:  1
Train size: 350, Validation size: 75, Test size: 75
Training CGCNN on CoRE2019 for Di...
Epoch: 1, Batch: 0, Loss: 0.0014862257521599531
Epoch: 1, Batch: 50, Loss: 0.04718615859746933
Epoch: 1, Batch: 100, Loss: 11.60198974609375
Epoch: 1, Batch: 150, Loss: 0.16312670707702637
Epoch: 1, Batch: 200, Loss: 0.000574486970435828
Epoch: 1, Batch: 250, Loss: 0.03360014408826828
Epoch: 1, Batch: 300, Loss: 0.16813799738883972
Validating model at epoch 1...
Epoch [1] Validate: [75/75], Loss 0.9027 (0.8340), MAE 3.510 (2.570)
MAE 2.570
Testing CGCNN on CoRE2019 for Di...
training_results/finetuning/CGCNN/CGCNN_scratch_CoRE2019_1_Di/checkpoints/model.pth
Loaded trained model with success.
Test: [74/75], Loss 0.1335 (0.5304), MAE 1.349 (2.271)
MAE 2.271


In [11]:
df.to_csv(
    os.path.join(log_dir, fn),
    mode='a', index=False, header=True
)